# Crawler für die DSGVO-Website

Die DSGVO kann in aufbereiteter Form auf der Webseite https://dsgvo-gesetz.de/ eingesehen werden. Hierbei handelt es sich um eine von intersoft consulting erstellte Webseite, die die DSGVO in einem einfachen Format präsentiert.

## Grundaufbau der Seiten

Das Grundgerüst aller Seiten folgt dem folgenden Schema:

```html
...
<body ...>
  <div id="page" ...>
    <div id="content" ...>
      <div id="primary" ...>
        <main id="main" ...>
          <article id="post-##" ...>
            <header class="entry-header">
              <h1 class="entry-title">
                <span class="dsgvo-number">[TITEL]</span>
                <span class="dsgvo-title">[UNTERTITEL]</span>
              </h1>
            </header> 
            <div class="entry-content">
              [INHALT]
            </div>  
          </article>
        </main>
      </div>
    </div>
  </div>
</body>
...
```

## Aufbau der Übersichtsseite

Die Übersichtsseite enthält einen kurzen erklärenden Text und danach Links zu den 11 Unterkapiteln sowie direkte Links zu den einzelnen Artikeln.

Der Titel der Seite ist "Datenschutz-Grundverordnung" und der Subtitel "DSGVO".

Die URL lautet: https://dsgvo-gesetz.de/

Der folgende Code zeigt den Inhalt der Seite, also des Platzhalters `[INHALT]`:

```html
::before
<p>...</p>
<h2>Schnellzugriff</h2>
<table ...>...</table>
<h2>Wichtige Themen</h2>
<ul class="beliebte inhalte">...</ul>
<div class="clear">...</div>
<h2>Inhaltsverzeichnis</h2>
<div class="liste-inhaltsuebersicht dsgvo">
  [VERZEICHNIS]
</div>
<div class="feedback hidden-print">...</div>
::after
```

Das Verzeichnis selbst ist wie folgt aufgebaut (Beispiel):

```html
<div class="kapitel nomargin">...</div>
<div class="artikel">...</div>
<div class="kapitel">...</div>
<div class="abschnitt">...</div>
<div class="artikel">...</div>
<div class="artikel">...</div>
<div class="abschnitt">...</div>
<div class="artikel">...</div>
<div class="artikel">...</div>
```

Dabei ist zu beachten, dass einzelne Kapitel entweder nur aus Artikeln bestehen (also keine Abschnitte enthalten), oder aus Abschnitten und Artikeln.

```html
<div class="kapitel">
  <span class="nummer">Kapitel 2</span>
  <span class="titel">Grundsätze</span>
</div>
```

```html
<div class="abschnitt">
  <span class="nummer">Abschnitt 1</span>
  <span class="titel">Transparenz und Modalitäten</span>
</div>
```

```html
<div class="artikel">
  <a href="https://dsgvo-gesetz.de/art-2-dsgvo/">
    <span class="nummer">Artikel 2</span>
    <span class="titel">Sachlicher Anwendungsbereich</span>
  </a>
</div>
```

Beim Crawlen der Seiten wird das Verzeichnis in der Übersichtsseite durchlaufen und die Links zu den einzelnen Artikeln ausgelesen. Dabei ist die Reihenfolge entscheidend, da durch diese klar wird, zu welchem Kapitel und ggf. Abschnitt ein Artikel gehört.

## Aufbau einer Artikelseite

URL eines Artikels: https://dsgvo-gesetz.de/art-##-dsgvo/

Der Abschnitt [INHALT] ist im einfachsten Fall wie folgt aufgaubaut, wie bspw. Artikel 10:

```html
<p>[TEXT]</p>
```

Meistens besteht ein Artikel aber aus mehreren Absätzen, welche als `(n)` nummeriert sind. Ein Beispiel dafür wäre Artikel 1. In diesem Fall wäre die Darstellung wie folgt:

```html
<ol>
  <li>[TEXT_ABSATZ_1]</li>
  <li>[TEXT_ABSATZ_2]</li>
  <li>[TEXT_ABSATZ_3]</li>
</ol>
```

Es ist ausßerdem möglich, dass ein Absatz in mehrere Unterabsätze unterteilt ist. Ein Beispiel wäre Artikel 2:

```html	
<ol>
  <li>[TEXT_ABSATZ_1]</li>
  <li>
    <p>[TEXT_UNTERABSATZ_0</p>
    <ol>
      <li>[TEXT_UNTERABSATZ_1]</li>
      <li>[TEXT_UNTERABSATZ_2]</li>
      <li>[TEXT_UNTERABSATZ_3]</li>
      <li>[TEXT_UNTERABSATZ_4]</li>
    <ol>  
  </li>
  <li>[TEXT_ABSATZ_3]</li>
  <li>[TEXT_ABSATZ_4]</li>
</ol>
```

Jeder der [TEXT...]-Platzhalter kann aus mehreren Sätzen bestehen. Falls dem so ist, wird jedem Satz ein `<sup>#</sup>` vorangestellt. Sollte nur ein Satz vorhanden sein, wird dieser ohne `<sup>#</sup>` dargestellt und es ist von Satz 1 auszugehen.

## Zielformat

Es soll eine JSONL-Datei erstellt werden, welche die Daten in einem strukturierten Format enthält:
- Kapitel_Nr
- Kapitel_Name
- Abschnitt_Nr
- Abschnitt_Name
- Artikel_Nr
- Artikel_Name
- Absatz_Nr (0 falls nicht explizit durch (#) angegeben)
- Unterabsatz_Nr (0 falls nicht explizit durch (a), (b), (c), ... angegeben, ansonsten a=1, b=2, c=3, ...)
- Satz_Nr (1 falls nicht explizit durch <sup>#</sup> angegeben)
- Text

## Ausgabe

Vom /src-Ordner ausgesehen:
../data/input/dsgvo_crawled.jsonl

In [16]:
from dataclasses import dataclass
from typing import Dict

@dataclass
class DSGVOEntry:
    """Repräsentiert einen DSGVO-Eintrag mit hierarchischer Struktur"""
    kapitel_nr: int
    kapitel_name: str
    abschnitt_nr: int       # 0 falls nicht vorhanden
    abschnitt_name: str     # "" falls nicht vorhanden
    artikel_nr: int
    artikel_name: str
    absatz_nr: int          # 0 falls nicht explizit durch (#) angegeben
    unterabsatz_nr: int     # 0 falls nicht explizit durch (a), (b), (c), ... angegeben, ansonsten a=1, b=2, c=3, ...
    satz_nr: int            # 1 falls nicht explizit durch <sup>#</sup> angegeben
    text: str

    def to_dict(self) -> Dict:
        """Konvertiert den Eintrag zu einem Dictionary für JSONL-Export"""
        return {
            'Kapitel_Nr': self.kapitel_nr,
            'Kapitel_Name': self.kapitel_name,
            'Abschnitt_Nr': self.abschnitt_nr,
            'Abschnitt_Name': self.abschnitt_name,
            'Artikel_nr': self.artikel_nr,
            'Artikel_Name': self.artikel_name,
            'Absatz_nr': self.absatz_nr,
            'Unterabsatz_nr': self.unterabsatz_nr,
            'Satz_nr': self.satz_nr,
            'Text': self.text
        }

In [17]:
import requests
import json
import logging
import time
from datetime import datetime
from pathlib import Path
from bs4 import BeautifulSoup
from typing import List, Tuple, Optional
import re

# Logging Setup
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('../data/logs/crawler.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)


In [18]:
class DSGVOCrawler:
    """DSGVO Website Crawler für https://dsgvo-gesetz.de/"""

    def __init__(self, base_url: str = "https://dsgvo-gesetz.de"):
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })

        # Struktur-Tracking
        self.current_kapitel = {"nr": 0, "name": ""}
        self.current_abschnitt = {"nr": 0, "name": ""}

    def get_page(self, url: str, retry_count: int = 3) -> Optional[BeautifulSoup]:
        """Lädt eine Seite mit Retry-Mechanismus"""
        for attempt in range(retry_count):
            try:
                logger.info(f"Lade Seite: {url} (Versuch {attempt + 1})")
                response = self.session.get(url, timeout=10)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'lxml')
                time.sleep(1)  # Rate limiting
                return soup

            except requests.RequestException as e:
                logger.warning(f"Fehler beim Laden von {url}: {e}")
                if attempt == retry_count - 1:
                    logger.error(f"Alle Versuche fehlgeschlagen für: {url}")
                    return None
                time.sleep(2 ** attempt)  # Exponential backoff

        return None

    def parse_overview_page(self) -> List[Tuple[str, str]]:
        """
        Parst die Übersichtsseite und extrahiert alle Artikel-URLs
        Returns: Liste von (artikel_url, artikel_context) Tupeln
        """
        soup = self.get_page(self.base_url)
        if not soup:
            logger.error("Konnte Übersichtsseite nicht laden")
            return []

        # Finde das Inhaltsverzeichnis
        inhaltsverzeichnis = soup.find('div', class_='liste-inhaltsuebersicht dsgvo')
        if not inhaltsverzeichnis:
            logger.error("Inhaltsverzeichnis nicht gefunden")
            return []

        artikel_links = []

        # Durchlaufe alle Elemente im Verzeichnis
        for element in inhaltsverzeichnis.find_all(['div']):
            class_name = element.get('class', [])

            if 'kapitel' in class_name:
                # Neues Kapitel
                nummer_span = element.find('span', class_='nummer')
                titel_span = element.find('span', class_='titel')

                if nummer_span and titel_span:
                    # Extrahiere Kapitelnummer
                    kapitel_text = nummer_span.get_text(strip=True)
                    kapitel_match = re.search(r'(\d+)', kapitel_text)
                    self.current_kapitel['nr'] = int(kapitel_match.group(1)) if kapitel_match else 0
                    self.current_kapitel['name'] = titel_span.get_text(strip=True)

                    # Reset Abschnitt
                    self.current_abschnitt = {"nr": 0, "name": ""}

                    logger.info(f"Gefunden: Kapitel {self.current_kapitel['nr']} - {self.current_kapitel['name']}")

            elif 'abschnitt' in class_name:
                # Neuer Abschnitt
                nummer_span = element.find('span', class_='nummer')
                titel_span = element.find('span', class_='titel')

                if nummer_span and titel_span:
                    # Extrahiere Abschnittsnummer
                    abschnitt_text = nummer_span.get_text(strip=True)
                    abschnitt_match = re.search(r'(\d+)', abschnitt_text)
                    self.current_abschnitt['nr'] = int(abschnitt_match.group(1)) if abschnitt_match else 0
                    self.current_abschnitt['name'] = titel_span.get_text(strip=True)

                    logger.info(f"Gefunden: Abschnitt {self.current_abschnitt['nr']} - {self.current_abschnitt['name']}")

            elif 'artikel' in class_name:
                # Artikel-Link
                link = element.find('a')
                if link and link.get('href'):
                    artikel_url = link.get('href')

                    # Kontext für diesen Artikel speichern
                    context = {
                        'kapitel_nr': self.current_kapitel['nr'],
                        'kapitel_name': self.current_kapitel['name'],
                        'abschnitt_nr': self.current_abschnitt['nr'],
                        'abschnitt_name': self.current_abschnitt['name']
                    }

                    artikel_links.append((artikel_url, context))

        logger.info(f"Insgesamt {len(artikel_links)} Artikel gefunden")
        return artikel_links

    def parse_article_page(self, url: str, context: dict) -> List[DSGVOEntry]:
        """
        Parst eine einzelne Artikelseite und extrahiert alle Einträge
        """
        soup = self.get_page(url)
        if not soup:
            logger.error(f"Konnte Artikel nicht laden: {url}")
            return []

        entries = []

        # Extrahiere Artikel-Info aus dem Header
        header = soup.find('header', class_='entry-header')
        if not header:
            logger.error(f"Header nicht gefunden in: {url}")
            return []

        # Artikel Nummer und Name
        dsgvo_number = header.find('span', class_='dsgvo-number')
        dsgvo_title = header.find('span', class_='dsgvo-title')

        if not dsgvo_number or not dsgvo_title:
            logger.error(f"Artikel-Info nicht gefunden in: {url}")
            return []

        artikel_text = dsgvo_number.get_text(strip=True)
        artikel_match = re.search(r'(\d+)', artikel_text)
        artikel_nr = int(artikel_match.group(1)) if artikel_match else 0
        artikel_name = dsgvo_title.get_text(strip=True)

        logger.info(f"Parse Artikel {artikel_nr}: {artikel_name}")

        # Extrahiere Inhalt
        content_div = soup.find('div', class_='entry-content')
        if not content_div:
            logger.error(f"Content nicht gefunden in: {url}")
            return []

        # Prüfe ob es sich um einen einfachen Artikel handelt (nur <p> Tags)
        paragraphs = content_div.find_all('p', recursive=False)
        if paragraphs and not content_div.find('ol', recursive=False):
            # Einfacher Artikel ohne nummerierte Absätze
            for p in paragraphs:
                text = p.get_text(strip=True)
                if text and not self._is_navigation_text(text):
                    # Prüfe auf Satz-Nummerierung mit <sup>
                    sentences = self._parse_sentences(p)
                    for satz_nr, satz_text in sentences:
                        entry = DSGVOEntry(
                            kapitel_nr=context['kapitel_nr'],
                            kapitel_name=context['kapitel_name'],
                            abschnitt_nr=context['abschnitt_nr'],
                            abschnitt_name=context['abschnitt_name'],
                            artikel_nr=artikel_nr,
                            artikel_name=artikel_name,
                            absatz_nr=0,  # Kein expliziter Absatz
                            unterabsatz_nr=0,  # Kein Unterabsatz
                            satz_nr=satz_nr,
                            text=satz_text
                        )
                        entries.append(entry)
        else:
            # Komplexer Artikel mit nummerierten Absätzen
            ol_elements = content_div.find_all('ol', recursive=False)
            for ol in ol_elements:
                entries.extend(self._parse_ordered_list(ol, context, artikel_nr, artikel_name))

        return entries

    def _parse_ordered_list(self, ol_element, context: dict, artikel_nr: int, artikel_name: str, parent_absatz: int = 0) -> List[DSGVOEntry]:
        """
        Parst eine <ol> Liste rekursiv für Absätze und Unterabsätze
        """
        entries = []

        for i, li in enumerate(ol_element.find_all('li', recursive=False), 1):
            absatz_nr = i if parent_absatz == 0 else parent_absatz

            # Prüfe ob es verschachtelte Listen gibt (Unterabsätze)
            nested_ol = li.find('ol')

            if nested_ol:
                # Dieser li hat Unterabsätze
                # Extrahiere Text vor der verschachtelten Liste
                li_copy = li.__copy__()
                nested_ol_copy = li_copy.find('ol')
                if nested_ol_copy:
                    nested_ol_copy.decompose()  # Entferne die verschachtelte Liste

                intro_text = li_copy.get_text(strip=True)
                if intro_text:
                    # Einleitungstext des Absatzes
                    sentences = self._parse_sentences_from_text(intro_text)
                    for satz_nr, satz_text in sentences:
                        entry = DSGVOEntry(
                            kapitel_nr=context['kapitel_nr'],
                            kapitel_name=context['kapitel_name'],
                            abschnitt_nr=context['abschnitt_nr'],
                            abschnitt_name=context['abschnitt_name'],
                            artikel_nr=artikel_nr,
                            artikel_name=artikel_name,
                            absatz_nr=absatz_nr,
                            unterabsatz_nr=0,  # Einleitungstext
                            satz_nr=satz_nr,
                            text=satz_text
                        )
                        entries.append(entry)

                # Parse Unterabsätze
                for j, sub_li in enumerate(nested_ol.find_all('li', recursive=False), 1):
                    text = sub_li.get_text(strip=True)
                    if text:
                        sentences = self._parse_sentences_from_text(text)
                        for satz_nr, satz_text in sentences:
                            entry = DSGVOEntry(
                                kapitel_nr=context['kapitel_nr'],
                                kapitel_name=context['kapitel_name'],
                                abschnitt_nr=context['abschnitt_nr'],
                                abschnitt_name=context['abschnitt_name'],
                                artikel_nr=artikel_nr,
                                artikel_name=artikel_name,
                                absatz_nr=absatz_nr,
                                unterabsatz_nr=j,  # a=1, b=2, c=3, ...
                                satz_nr=satz_nr,
                                text=satz_text
                            )
                            entries.append(entry)
            else:
                # Normaler Absatz ohne Unterabsätze
                text = li.get_text(strip=True)
                if text:
                    sentences = self._parse_sentences_from_text(text)
                    for satz_nr, satz_text in sentences:
                        entry = DSGVOEntry(
                            kapitel_nr=context['kapitel_nr'],
                            kapitel_name=context['kapitel_name'],
                            abschnitt_nr=context['abschnitt_nr'],
                            abschnitt_name=context['abschnitt_name'],
                            artikel_nr=artikel_nr,
                            artikel_name=artikel_name,
                            absatz_nr=absatz_nr,
                            unterabsatz_nr=0,
                            satz_nr=satz_nr,
                            text=satz_text
                        )
                        entries.append(entry)

        return entries

    def _parse_sentences(self, element) -> List[Tuple[int, str]]:
        """
        Parst Sätze aus einem HTML-Element und erkennt <sup> Nummerierung
        """
        sentences = []

        # Prüfe auf <sup> Tags für Satz-Nummerierung
        sup_tags = element.find_all('sup')

        if sup_tags:
            # Es gibt nummerierte Sätze
            for sup in sup_tags:
                satz_nr_text = sup.get_text(strip=True)
                try:
                    satz_nr = int(satz_nr_text)
                except ValueError:
                    satz_nr = 1

                # Extrahiere Text nach dem <sup> Tag bis zum nächsten <sup> oder Ende
                text_parts = []
                current = sup.next_sibling

                while current and (not hasattr(current, 'name') or current.name != 'sup'):
                    if hasattr(current, 'get_text'):
                        text_parts.append(current.get_text())
                    else:
                        text_parts.append(str(current))
                    current = current.next_sibling

                satz_text = ''.join(text_parts).strip()
                if satz_text:
                    sentences.append((satz_nr, satz_text))
        else:
            # Kein <sup> Tag gefunden, gesamter Text ist Satz 1
            text = element.get_text(strip=True)
            if text:
                sentences.append((1, text))

        return sentences

    def _parse_sentences_from_text(self, text: str) -> List[Tuple[int, str]]:
        """
        Parst Sätze aus einem Text-String (fallback für bereits extrahierten Text)
        """
        # Einfache Implementation: Text ist ein Satz
        if text.strip():
            return [(1, text.strip())]
        return []

    def _is_navigation_text(self, text: str) -> bool:
        """
        Prüft ob es sich um Navigations- oder Meta-Text handelt
        """
        navigation_keywords = [
            'feedback', 'bewertung', 'drucken', 'teilen',
            'weitere artikel', 'siehe auch', 'navigation'
        ]
        text_lower = text.lower()
        return any(keyword in text_lower for keyword in navigation_keywords)

    def crawl_all_articles(self) -> List[DSGVOEntry]:
        """
        Hauptmethode: Crawlt alle Artikel und gibt strukturierte Daten zurück
        """
        logger.info("Starte DSGVO Crawler...")

        # 1. Parse Übersichtsseite für Artikel-Links
        artikel_links = self.parse_overview_page()
        if not artikel_links:
            logger.error("Keine Artikel gefunden")
            return []

        all_entries = []

        # 2. Parse jeden Artikel
        for i, (artikel_url, context) in enumerate(artikel_links, 1):
            logger.info(f"Verarbeite Artikel {i}/{len(artikel_links)}: {artikel_url}")

            try:
                entries = self.parse_article_page(artikel_url, context)
                all_entries.extend(entries)
                logger.info(f"Artikel {i}: {len(entries)} Einträge extrahiert")

            except Exception as e:
                logger.error(f"Fehler beim Verarbeiten von {artikel_url}: {e}")
                continue

        logger.info(f"Crawling abgeschlossen. Insgesamt {len(all_entries)} Einträge extrahiert.")
        return all_entries

    def save_to_jsonl(self, entries: List[DSGVOEntry], output_path: str):
        """
        Speichert die Einträge als JSONL-Datei
        """
        # Erstelle Output-Verzeichnis falls nicht vorhanden
        output_file = Path(output_path)
        output_file.parent.mkdir(parents=True, exist_ok=True)

        logger.info(f"Speichere {len(entries)} Einträge in: {output_path}")

        with open(output_path, 'w', encoding='utf-8') as f:
            for entry in entries:
                json_line = json.dumps(entry.to_dict(), ensure_ascii=False)
                f.write(json_line + '\n')

        logger.info(f"Erfolgreich gespeichert: {output_path}")

    def run_full_crawl(self, output_path: str = None):
        """
        Führt einen vollständigen Crawl-Durchlauf aus
        """
        if output_path is None:
            timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
            output_path = f"../data/output/dsgvo_crawled_{timestamp}.jsonl"

        try:
            # Crawle alle Artikel
            entries = self.crawl_all_articles()

            if entries:
                # Speichere Ergebnisse
                self.save_to_jsonl(entries, output_path)

                # Statistiken
                logger.info("=== CRAWLING STATISTIKEN ===")
                logger.info(f"Gesamt Einträge: {len(entries)}")

                # Gruppiere nach Artikeln
                artikel_count = len(set((e.artikel_nr for e in entries)))
                logger.info(f"Artikel verarbeitet: {artikel_count}")

                # Gruppiere nach Kapiteln
                kapitel_count = len(set((e.kapitel_nr for e in entries)))
                logger.info(f"Kapitel verarbeitet: {kapitel_count}")

                return output_path
            else:
                logger.error("Keine Einträge zum Speichern gefunden")
                return None

        except Exception as e:
            logger.error(f"Fehler beim Crawling: {e}")
            return None


In [19]:
# Beispiel-Verwendung und Test des Crawlers

if __name__ == "__main__":
    # Erstelle Crawler-Instanz
    crawler = DSGVOCrawler()

    # Führe vollständigen Crawl aus
    output_file = crawler.run_full_crawl()

    if output_file:
        print(f"\\n✅ Crawling erfolgreich abgeschlossen!")
        print(f"📄 Ausgabedatei: {output_file}")
        print(f"📁 Log-Datei: ../data/logs/crawler.log")

        # Kurze Vorschau der ersten Einträge
        with open(output_file, 'r', encoding='utf-8') as f:
            lines = f.readlines()[:3]
            print(f"\\n📋 Erste {len(lines)} Einträge:")
            for i, line in enumerate(lines, 1):
                entry = json.loads(line)
                print(f"{i}. Artikel {entry['Artikel_nr']}, Absatz {entry['Absatz_nr']}, Satz {entry['Satz_nr']}")
                print(f"   Text: {entry['Text'][:100]}...")
    else:
        print("❌ Crawling fehlgeschlagen!")


2025-08-11 12:04:44,762 - INFO - Starte DSGVO Crawler...
2025-08-11 12:04:44,763 - INFO - Lade Seite: https://dsgvo-gesetz.de (Versuch 1)
2025-08-11 12:04:45,989 - INFO - Gefunden: Kapitel 1 - Allgemeine Bestimmungen
2025-08-11 12:04:45,991 - INFO - Gefunden: Kapitel 2 - Grundsätze
2025-08-11 12:04:45,994 - INFO - Gefunden: Kapitel 3 - Rechte der betroffenen Person
2025-08-11 12:04:45,997 - INFO - Gefunden: Abschnitt 1 - Transparenz und Modalitäten
2025-08-11 12:04:45,999 - INFO - Gefunden: Abschnitt 2 - Informationspflicht und Recht auf Auskunft zu personenbezogenen Daten
2025-08-11 12:04:46,002 - INFO - Gefunden: Abschnitt 3 - Berichtigung und Löschung
2025-08-11 12:04:46,004 - INFO - Gefunden: Abschnitt 4 - Widerspruchsrecht und automatisierte Entscheidungsfindung im Einzelfall
2025-08-11 12:04:46,007 - INFO - Gefunden: Abschnitt 5 - Beschränkungen
2025-08-11 12:04:46,009 - INFO - Gefunden: Kapitel 4 - Verantwortlicher und Auftragsverarbeiter
2025-08-11 12:04:46,010 - INFO - Gefunde

\n✅ Crawling erfolgreich abgeschlossen!
📄 Ausgabedatei: ../data/output/dsgvo_crawled_2025-08-11_1204.jsonl
📁 Log-Datei: ../data/logs/crawler.log
\n📋 Erste 3 Einträge:
1. Artikel 1, Absatz 1, Satz 1
   Text: Diese Verordnung enthält Vorschriften zum Schutz natürlicher Personen bei der Verarbeitung personenb...
2. Artikel 1, Absatz 2, Satz 1
   Text: Diese Verordnung schützt die Grundrechte und Grundfreiheiten natürlicher Personen und insbesondere d...
3. Artikel 1, Absatz 3, Satz 1
   Text: Der freie Verkehr personenbezogener Daten in der Union darf aus Gründen des Schutzes natürlicher Per...
